In [ ]:
import polars as pl

import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2
df = pl.read_parquet("pile.parquet")

In [ ]:
df_sampled = df.sample(100)
for row in df_sampled.iter_rows(named=True):
    print(row['text'])
